<a href="https://colab.research.google.com/github/HasnathBeevi/TextGuard/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split


from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')
from nltk.stem import PorterStemmer

porter = PorterStemmer()
from google.colab import files
uploaded = files.upload()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving spam.csv to spam (5).csv


In [2]:
df = pd.read_csv('spam.csv', encoding='latin-1')
df.head()

#df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.rename(columns={'v1': 'Class', 'v2': 'Text'}, inplace=True)
df['Class'] = df['Class'].map({'ham':0, 'spam':1})





stop_words = stopwords.words('english')
print(stop_words[::10])



def clean_text(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ", words)
    text = words.lower().split()
    return " ".join(text)

def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop_words]
    return " ".join(text)

def stemmer(stem_text):
    """The function to apply stemming"""
    stem_text = [porter.stem(word) for word in stem_text.split()]
    return " ".join(stem_text)


df['Text'] = df['Text'].apply(clean_text)
df['Text'] = df['Text'].apply(remove_stopwords)
df['Text'] = df['Text'].apply(stemmer)


X = df['Text']
y = df['Class']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

['i', "you've", 'himself', 'they', 'that', 'been', 'a', 'while', 'through', 'in', 'here', 'few', 'own', 'just', 're', 'doesn', 'ma', "shouldn't"]


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Vectorizing the text data using TfidfVectorizer
vectorizer = TfidfVectorizer()

# Creating a pipeline that first transforms the text using Tfidf and then applies Logistic Regression
model = Pipeline([
    ('tfidf', vectorizer),
    ('clf', LogisticRegression(max_iter=1000))
])

# Training the model
model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Displaying the classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.96
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       949
           1       0.99      0.74      0.85       166

    accuracy                           0.96      1115
   macro avg       0.97      0.87      0.91      1115
weighted avg       0.96      0.96      0.96      1115



In [4]:
import joblib

# Save the model to a file
joblib.dump(model, 'sms_spam_classifier.pkl')


['sms_spam_classifier.pkl']

In [5]:
from google.colab import files
uploaded = files.upload()
df_test = pd.read_csv('sms_messages_labelled.csv', encoding='latin-1')
df_test.head()

#df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df_test.rename(columns={'v1': 'Class', 'v2': 'Text'}, inplace=True)
df_test['Class'] = df_test['Class'].map({'ham':0, 'spam':1})





stop_words = stopwords.words('english')
print(stop_words[::10])



def clean_text(words):
    """The function to clean text"""
    words = re.sub("[^a-zA-Z]"," ", words)
    text = words.lower().split()
    return " ".join(text)

def remove_stopwords(text):
    """The function to removing stopwords"""
    text = [word.lower() for word in text.split() if word.lower() not in stop_words]
    return " ".join(text)

def stemmer(stem_text):
    """The function to apply stemming"""
    stem_text = [porter.stem(word) for word in stem_text.split()]
    return " ".join(stem_text)


df_test['Text'] = df_test['Text'].apply(clean_text)
df_test['Text'] = df_test['Text'].apply(remove_stopwords)
df_test['Text'] = df_test['Text'].apply(stemmer)


X_new= df_test['Text']
y_new = df_test['Class']

X_train_new,X_test_new,y_train_new,y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state = 0)



Saving sms_messages_labelled.csv to sms_messages_labelled (7).csv
['i', "you've", 'himself', 'they', 'that', 'been', 'a', 'while', 'through', 'in', 'here', 'few', 'own', 'just', 're', 'doesn', 'ma', "shouldn't"]


In [6]:

# Load the model from the file
loaded_model = joblib.load('sms_spam_classifier.pkl')

# Now you can use loaded_model to make predictions

# Make predictions on the test data
y_pred_new = loaded_model.predict(X_test_new)


In [10]:
from sklearn.metrics import classification_report, accuracy_score

if y_test is not None:
    # Calculate accuracy
    accuracy_new = accuracy_score(y_test_new, y_pred_new)
    print(f'Accuracy on new test data: {accuracy_new:.2f}')

    # Print classification report
    print(classification_report(y_test_new, y_pred_new))
else:
    print("Predictions on the new test data:")
    print(y_pred_new)


Accuracy on new test data: 0.49
              precision    recall  f1-score   support

         0.0       0.47      0.95      0.63        20
         1.0       0.67      0.09      0.15        23

    accuracy                           0.49        43
   macro avg       0.57      0.52      0.39        43
weighted avg       0.58      0.49      0.38        43



In [13]:
# Set up Git username and email
!git config --global user.name "HasnathBeevi"
!git config --global user.email "hasnathhamza@gmail.com"


In [14]:
# Clone the repository (replace with your repository URL)
!git clone https://github.com/HasnathBeevi/TextGuard

# Navigate to the cloned repository folder
%cd TextGuard


Cloning into 'TextGuard'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 7.75 KiB | 1.94 MiB/s, done.
/content/TextGuard
